In [1]:
import pandas as pd 
df_club=pd.read_csv("Clubs_data.csv")
df_meta=pd.read_csv("Metadata.csv")
df_organiser=pd.read_csv("Organisers_In_Fests.csv")
df_prarticipants=pd.read_csv("Participants_In_Fests.csv")

In [2]:
df_club.head()

,Club_Name,Name,Event,Role
0,club_3,Darien Ma,club_3_event_2,Participant
1,club_2,Sege Schrock,club_2_event_1,Participant
2,club_3,Alison bci,club_3_event_2,Participant
3,club_1,kiara jauregui,club_1_event_1,Participant
4,club_2,Tete Buim,club_2_event_3,Participant


In [3]:
df_meta.head()

,Name,ID
0,Annette Ahmed,18XJ1A0100
1,Zachery Clancy,18XJ1A0101
2,Jaquelin Duran,18XJ1A0102
3,Orlando Clemens,18XJ1A0103
4,Dorothy Mayhew,18XJ1A0104


In [4]:
df_organiser.head()

,Fest_Name,Name,Role
0,fest_2,Stcerr Hbmond,organiser_2
1,fest_2,Ayliwn Slks,organiser_6
2,fest_2,Sebestien Hickjman,organiser_7
3,fest_2,Grechen Chlewsn,organiser_8
4,fest_1,Evelin B,organiser_1


In [5]:
df_prarticipants.head()

,Fest_Name,Name,Event
0,fest_1,JenapRowli,fest_1_event_9
1,fest_1,Credyj Duenas,fest_1_event_4
2,fest_2,Jequqlin Duran,fest_2_event_1
3,fest_1,Keiyah widalgo,fest_1_event_4
4,fest_2,Elaine John S,fest_2_event_15
